# Importing Modules

In [ ]:
import numpy as np 
import pandas as pd 
import os
from lightgbm import LGBMRegressor
import matplotlib.pyplot
from sklearn.model_selection import TimeSeriesSplit

# Loading Data

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Creating a Data Frame

In [ ]:

train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
train[['pred_carbon_monoxide','pred_benzen','pred_nitrogen_oxides']] =0,0,0
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
ss= pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
#Plot sensors 2-4 for first 5 days
#reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html
test.iloc[:5*24,:].plot(y=['sensor_2', 'sensor_3', 'sensor_2'], 
                        title = 'Daily Sensor data', xlabel='Days', ylabel='Readings')

In [ ]:

#Plot Mean of all Sensors
#reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html
test.mean().plot(y=['sensor_2', 'sensor_3', 'sensor_2'], 
                        title = 'Mean Values', kind = 'bar')

In [ ]:

i=0;j=72;k=5
train[[f'sensor_{i}' for i in range(1,4)]].iloc[((24)*i + j):(24*(i+k) + j)].plot()

In [ ]:

def SCORE_FUNCTION(preds, truth):
    return np.mean(np.square(np.log1p(preds) - np.log1p(truth)))

#5x Crossval
model = LGBMRegressor()
tss = TimeSeriesSplit()
FEATURES = ['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']

scores = []
for train_idx, test_idx in tss.split(train):
    score = []
    for TARGET in ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']:
        predictor = '_'.join(['pred'] + TARGET.split('_')[1:])
        
        model.fit(train[FEATURES].iloc[train_idx], train[[TARGET]].iloc[train_idx])
        preds = np.clip(model.predict(train[FEATURES].iloc[test_idx]), a_min=0., a_max=1e100)
        score.append(SCORE_FUNCTION(preds, train[TARGET].iloc[test_idx].values))
     
    score.append(np.mean(score))
    scores.append(pd.DataFrame(score).T)
    print(f'{score[0] :.4f}, {score[1] :.4f}, {score[2] :.4f}, {score[3] :.4f}')
    

#Predictions
for TARGET in ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']:
    model.fit(train[FEATURES], train[[TARGET]])
    ss[TARGET] = np.clip(model.predict(test[FEATURES]), 0., 1e10)


ss.to_csv('submission.csv', index=False)